In [1]:
import os
import sys
%load_ext autoreload
%autoreload 2
import pandas as pd

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from vadbp.variant_comparator import VariantComparator, DataType

In [3]:
def clean_df(df):

    explode_df = df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)
    explode_df['time:timestamp'] = explode_df[['intime', 'outtime']].values.tolist()
    explode_df['a'] = 'start'
    explode_df['b'] = 'complete'
    explode_df['lifecycle:transition'] = explode_df[['a', 'b']].values.tolist()
    explode_df = explode_df.explode(['time:timestamp', 'lifecycle:transition'])
    explode_df.drop(['intime', 'outtime', 'a', 'b'], axis=1 ,inplace=True)
    explode_df['concept:name'] = explode_df['department']
    explode_df['case:concept:name'] = explode_df['subject_id']
    return explode_df

In [4]:
csv_data = pd.read_csv('../Event Logs_2nd_Iteration/Heart_Failure_Log.csv')
csv_data = clean_df(csv_data)
csv_data

,subject_id,hadm_id,transfer_id,department,Glucose,Abnormal Glucose,Potassium,Abnormal Potassium,Sodium,Abnormal Sodium,...,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,time:timestamp,lifecycle:transition,concept:name,case:concept:name
0,10559183,20008400.0,31433513,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2116-01-03 13:32:00,start,Emergency Department,10559183
0,10559183,20008400.0,31433513,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2116-01-03 16:00:00,complete,Emergency Department,10559183
1,10559183,20008400.0,36107959,Cardiac ICU,164.000000,1.0,4.316667,1.0,134.833333,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2116-01-03 16:00:00,start,Cardiac ICU,10559183
1,10559183,20008400.0,36107959,Cardiac ICU,164.000000,1.0,4.316667,1.0,134.833333,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2116-01-06 18:44:57,complete,Cardiac ICU,10559183
2,10559183,20008400.0,38390977,Post-ICU Cardiology,125.000000,1.0,4.300000,2.0,137.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2116-01-06 18:44:57,start,Post-ICU Cardiology,10559183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4320,11282384,29994261.0,30592303,Surgical ICU,138.833333,1.0,4.000000,2.0,142.333333,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2124-09-21 18:35:27,complete,Surgical ICU,11282384
4321,11282384,29994261.0,37585480,Post-ICU Medicine,114.333333,1.0,4.466667,2.0,142.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2124-09-21 18:35:27,start,Post-ICU Medicine,11282384
4321,11282384,29994261.0,37585480,Post-ICU Medicine,114.333333,1.0,4.466667,2.0,142.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2124-09-24 13:13:59,complete,Post-ICU Medicine,11282384
4322,11282384,29994261.0,39587256,Discharged,123.000000,1.0,4.300000,2.0,140.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2124-09-24 13:13:59,start,Discharged,11282384


In [5]:
# dataset split
subject_over_60  = csv_data[csv_data['age']>60]['subject_id']
subject_under_60  = csv_data[csv_data['age']<=60]['subject_id']
over_60 = csv_data[csv_data['subject_id'].isin(subject_over_60)]
under_60 = csv_data[csv_data['subject_id'].isin(subject_under_60)]

In [45]:
over_60 = over_60[over_60['lifecycle:transition'] == 'start']
over_60 = over_60.sort_values(['case:concept:name', 'time:timestamp'])
over_60

,subject_id,hadm_id,transfer_id,department,Glucose,Abnormal Glucose,Potassium,Abnormal Potassium,Sodium,Abnormal Sodium,...,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,time:timestamp,lifecycle:transition,concept:name,case:concept:name
4237,10010471,29842315.0,31660580,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,s/p cardiac arrest,2155-12-02 16:00:00,start,Emergency Department,10010471
4238,10010471,29842315.0,32119961,Cardiac ICU,126.666667,1.0,5.2,1.0,134.5,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2155-12-02 20:33:00,start,Cardiac ICU,10010471
4239,10010471,29842315.0,37777199,Discharged,138.000000,1.0,5.4,1.0,136.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2155-12-07 18:19:18,start,Discharged,10010471
1855,10013015,24173031.0,34801291,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,...,98.0,134.0,63.0,0.0,1.0,"Bradycardia, Transfer",2121-07-27 23:40:00,start,Emergency Department,10013015
1856,10013015,24173031.0,30738148,Cardiac ICU,165.000000,1.0,4.7,2.0,144.5,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2121-07-28 01:11:00,start,Cardiac ICU,10013015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3226,19994379,27334101.0,37289217,Discharged,93.000000,2.0,3.9,2.0,136.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2131-06-21 17:25:05,start,Discharged,19994379
1329,19998330,23151993.0,34983932,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,200.0,69.0,0.0,2.0,SOB,2178-09-20 17:43:00,start,Emergency Department,19998330
1330,19998330,23151993.0,31085771,Medical ICU,275.000000,1.0,4.7,2.0,140.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2178-09-20 21:40:00,start,Medical ICU,19998330
1331,19998330,23151993.0,39603320,Post-ICU Medicine,151.500000,1.0,4.8,2.0,139.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2178-09-21 18:06:26,start,Post-ICU Medicine,19998330


In [49]:
joined = over_60.join(over_60.shift(-1), lsuffix='_1', rsuffix='_2')
joined = joined[joined['case:concept:name_1'] == joined['case:concept:name_2']]
joined

,subject_id_1,hadm_id_1,transfer_id_1,department_1,Glucose_1,Abnormal Glucose_1,Potassium_1,Abnormal Potassium_1,Sodium_1,Abnormal Sodium_1,...,o2sat_2,sbp_2,dbp_2,pain_2,acuity_2,chiefcomplaint_2,time:timestamp_2,lifecycle:transition_2,concept:name_2,case:concept:name_2
4237,10010471,29842315.0,31660580,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2155-12-02 20:33:00,start,Cardiac ICU,10010471.0
4238,10010471,29842315.0,32119961,Cardiac ICU,126.666667,1.0,5.200000,1.0,134.500000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2155-12-07 18:19:18,start,Discharged,10010471.0
1855,10013015,24173031.0,34801291,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2121-07-28 01:11:00,start,Cardiac ICU,10013015.0
1856,10013015,24173031.0,30738148,Cardiac ICU,165.000000,1.0,4.700000,2.0,144.500000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2121-07-28 22:01:15,start,Post-ICU Cardiology,10013015.0
1857,10013015,24173031.0,37403933,Post-ICU Cardiology,95.055556,1.0,4.616667,2.0,140.944444,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2121-08-12 16:37:46,start,Discharged,10013015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3224,19994379,27334101.0,33110615,Surgical ICU,90.333333,2.0,3.866667,2.0,129.000000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2131-06-07 22:43:21,start,Post-ICU Cardiology,19994379.0
3225,19994379,27334101.0,30335411,Post-ICU Cardiology,112.272727,1.0,3.836364,1.0,137.500000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2131-06-21 17:25:05,start,Discharged,19994379.0
1329,19998330,23151993.0,34983932,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2178-09-20 21:40:00,start,Medical ICU,19998330.0
1330,19998330,23151993.0,31085771,Medical ICU,275.000000,1.0,4.700000,2.0,140.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2178-09-21 18:06:26,start,Post-ICU Medicine,19998330.0


In [50]:


df_aggregated = joined.groupby(['concept:name_1', 'concept:name_2']).agg(lambda x: x.dropna().tolist())
df_melted = pd.melt(df_aggregated.reset_index(), id_vars=['concept:name_1', 'concept:name_2'],
value_vars=df_aggregated.columns)
df_final = df_melted[df_melted['value'].map(lambda d: len(d)) > 0]
df_final

,concept:name_1,concept:name_2,variable,value
0,Cardiac ICU,Discharged,subject_id_1,"[10010471, 10106899, 10196650, 10274368, 10292..."
1,Cardiac ICU,Post-ICU Cardiology,subject_id_1,"[10013015, 10051074, 10182211, 10303080, 10357..."
2,Cardiac ICU,Post-ICU Medicine,subject_id_1,"[10316237, 10327156, 10637554, 10841600, 11065..."
3,Cardiac ICU,Post-ICU Surgery,subject_id_1,"[10564547, 14098557, 14909918, 15081932, 15615..."
4,Discharged,Emergency Department,subject_id_1,"[10108435, 10564547, 10684958, 10684958, 10839..."
...,...,...,...,...
6187,Pre-ICU Surgery,Surgical ICU,case:concept:name_2,"[10373824.0, 13593993.0, 16525301.0, 19879454.0]"
6188,Surgical ICU,Discharged,case:concept:name_2,"[10483660.0, 10606352.0, 12508471.0, 12730395...."
6189,Surgical ICU,Post-ICU Cardiology,case:concept:name_2,"[10714454.0, 11021643.0, 11058391.0, 11132535...."
6190,Surgical ICU,Post-ICU Medicine,case:concept:name_2,"[10048899.0, 10078008.0, 10108435.0, 10332328...."


In [79]:
a1 = df_final[df_final['variable'].str.endswith('_1')]
a1['variable'] = a1['variable'].apply(lambda row: row.rstrip('_1'))
a2 = df_final[df_finaal['variable'].str.endswith('_2')]
print(f"a1 len = {len(a1)} and a2 len = {len(a2)}")
a2['variable'] = a2['variable'].apply(lambda row: row.rstrip('_2'))
merged = a1.merge(a2, on=['variable', 'concept:name_1', 'concept:name_2'], suffixes=['_l', '_r'])
merged = merged.set_index(['concept:name_1', 'concept:name_2', 'variable'])
merged

a1 len = 1965 and a2 len = 2198


/var/folders/2z/hk7x37vj029gb48kv3xv2_hh0000gn/T/ipykernel_82801/1669893985.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a1['variable'] = a1['variable'].apply(lambda row: row.rstrip('_1'))
/var/folders/2z/hk7x37vj029gb48kv3xv2_hh0000gn/T/ipykernel_82801/1669893985.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a2['variable'] = a2['variable'].apply(lambda row: row.rstrip('_2'))


value_l  \
concept:name_1  concept:name_2       variable                                                               
Cardiac ICU     Discharged           subject_id         [10010471, 10106899, 10196650, 10274368, 10292...   
                Post-ICU Cardiology  subject_id         [10013015, 10051074, 10182211, 10303080, 10357...   
                Post-ICU Medicine    subject_id         [10316237, 10327156, 10637554, 10841600, 11065...   
                Post-ICU Surgery     subject_id         [10564547, 14098557, 14909918, 15081932, 15615...   
Discharged      Emergency Department subject_id         [10108435, 10564547, 10684958, 10684958, 10839...   
...                                                                                                   ...   
Pre-ICU Surgery Surgical ICU         case:concept:name           [10373824, 13593993, 16525301, 19879454]   
Surgical ICU    Discharged           case:concept:name  [10483660, 10606352, 12508471, 12730395, 12815...   
                Post-ICU Cardiology  case:concept:name  [10714454, 11021643, 11058391, 11132535, 11390...   
                Post-ICU Medicine    case:concept:name  [10048899, 10078008, 10108435, 10332328, 10335...   
                Post-ICU Surgery     case:concept:name  [10329919, 10373824, 13143572, 13337031, 13593...   

                                                                                                  value_r  
concept:name_1  concept:name_2       variable                                                              
Cardiac ICU     Discharged           subject_id         [10010471.0, 10106899.0, 10196650.0, 10274368....  
                Post-ICU Cardiology  subject_id         [10013015.0, 10051074.0, 10182211.0, 10303080....  
                Post-ICU Medicine    subject_id         [10316237.0, 10327156.0, 10637554.0, 10841600....  
                Post-ICU Surgery     subject_id         [10564547.0, 14098557.0, 14909918.0, 15081932....  
Discharged      Emergency Department subject_id         [10108435.0, 10564547.0, 10684958.0, 10684958....  
...                                                                                                   ...  
Pre-ICU Surgery Surgical ICU         case:concept:name   [10373824.0, 13593993.0, 16525301.0, 19879454.0]  
Surgical ICU    Discharged           case:concept:name  [10483660.0, 10606352.0, 12508471.0, 12730395....  
                Post-ICU Cardiology  case:concept:name  [10714454.0, 11021643.0, 11058391.0, 11132535....  
                Post-ICU Medicine    case:concept:name  [10048899.0, 10078008.0, 10108435.0, 10332328....  
                Post-ICU Surgery     case:concept:name  [10329919.0, 10373824.0, 13143572.0, 13337031....  

[1644 rows x 2 columns]

array(['subject_id', 'hadm_id', 'transfer_id', 'department', 'Glucose',
       'Abnormal Glucose', 'Potassium', 'Abnormal Potassium', 'Sodium',
       'Abnormal Sodium', 'Chloride', 'Abnormal Chloride', 'Creatinine',
       'Abnormal Creatinine', 'Urea Nitrogen', 'Abnormal Urea Nitrogen',
       'Bicarbonate', 'Abnormal Bicarbonate', 'Anion Gap',
       'Abnormal Anion Gap', 'Magnesium', 'Abnormal Magnesium',
       'Phosphate', 'Abnormal Phosphate', 'Calcium', 'Abnormal Calcium',
       'Hematocrit', 'Abnormal Hematocrit', 'Hemoglobin',
       'Abnormal Hemoglobin', 'Platelet Count', 'Abnormal Platelet Count',
       'White Blood Cells', 'Abnormal White Blood Cells', 'MCV',
       'Abnormal MCV', 'MCHC', 'Abnormal MCHC', 'MCH', 'Abnormal MCH',
       'Red Blood Cells', 'Abnormal Red Blood Cells', 'RDW',
       'Abnormal RDW', 'PTT', 'Abnormal PTT', 'INR(PT)',
       'Abnormal INR(PT)', 'PT', 'Abnormal PT', 'pH', 'Abnormal pH',
       'RDW-SD', 'Abnormal RDW-SD', 'pO', 'Abnormal pO',
 